In [ ]:
# Name, has a title
def get_title(sample):
    title = []
    for i in range(len(sample.Name)):
        tmp = sample.Name[i].split(',')
        title.append(tmp[-1].split('.')[0]) #or to combine use regexp and search
    sample['Title'] = title
    return sample

def get_titlekey(sample):
    title = []
    dict_title = {
    "Capt":       0,
    "Col":        0,
    "Major":      0,
    "Jonkheer":   1,
    "Don":        1,
    "Sir" :       1,
    "Dr":         0,
    "Rev":        0,
    "the Countess":1,
    "Dona":       1,
    "Mme":        2,
    "Mlle":       3,
    "Ms":         2,
    "Mr" :        4,
    "Mrs" :       2,
    "Miss" :      3,
    "Master" :    5,
    "Lady" :      1
}
    
    title_key = []
    for i in range(len(sample.Title)):
        tmp = sample.Title[i].strip()
        if tmp in dict_title.keys():
            title_key.append(dict_title[tmp])
        
    sample['TitleKey'] = title_key
    return sample

def get_surname(sample):
    Surname = []
    for n in range(len(sample)):
        Surname.append(sample.Name[n].split(',')[0])
    sample['Surname'] = Surname

    from collections import Counter
    traveling_partner = Counter(Surname)

    Family = []
    for n in range(len(Surname)):
        Family.append(traveling_partner.get(Surname[n]))
    sample['Family'] = Family
    return sample

def transform_sex(sample):
    for i in range(len(sample.Sex)):
        if sample.Age[i] < 8:
            sample.Sex[i] = 'child'
    return sample

# Group continuous data to prevent overfitting
def simplify_ages(sample):
    import pandas
    sample.Age = sample.Age.fillna(-0.5)
    bins = (-1, 0,  12 , 18, 35 , 60, 100)
    group_names = ['Unknown', 'Child', 'Teenager', 'Young Adult', 'Adult', 'Senior']
    categories = pandas.cut(sample.Age, bins, labels=group_names) #puts things in bins, this is great
    sample.Age = categories
    return sample


def simplify_cabins(sample):
    sample.Cabin = sample.Cabin.fillna('N')
    sample.Cabin = sample.Cabin.apply(lambda s: s[0])
    return sample

def simplify_fare(sample):
    import pandas as pd
    sample.Fare = sample.Fare.fillna(sample.Fare.median())
    sample.Fare.astype(int)
    bins = (-1,0,8,15,31,1000) #Picked from describe values
    group_names = ['Unknown','First','Second','Third','Fourth']
    categories = pd.cut(sample.Fare,bins,labels = group_names)
    sample.Fare = categories
    return sample


def transform_features(sample):
    get_title(sample)
    get_titlekey(sample)
    get_surname(sample)
    transform_sex(sample)
    simplify_ages(sample)
    simplify_cabins(sample)
    simplify_fare(sample)
   
    return sample


#Variables should be numerical and standardized for ML algorithms
def encode_features(X):
    from sklearn import preprocessing 
    features = ['Sex','Fare','Cabin','Title','Age','Surname']
    for feature in features:
        le = preprocessing.LabelEncoder()
        le = le.fit(X[feature].astype('category'))
        X[feature] = le.transform(X[feature])
        
    feature_mappings = {index: label for index, label in 
                  enumerate(le.classes_)}
    

    return X, feature_mappings



